# Find mismatches

Source files are created from the APPLAUSE database by this query:

```
SELECT * 
FROM applause_dr4.source 
WHERE plate_id = 19019
ORDER BY source_id
```
with another separate call for the source_calib table.

We look for sources that show up in two separate plates. The ones that don't are the candidates we should look in more detail.

The script exploits parallelism to expedite the search.

In [1]:
import os, io
import csv
from math import sqrt

import multiprocessing as mp
from multiprocessing import Pool

import numpy as np

from astropy import units as u
from astropy.io import fits
from astropy.io.fits import Header, Card
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Longitude, Latitude

from regions import PolygonSkyRegion
from mocpy import MOC

from settings import get_parameters, fname, current_dataset
from library import Worker, Worker2, is_in_jupyter

In [2]:
in_jupyter = is_in_jupyter()

print(in_jupyter)

True


In [3]:
par = get_parameters(current_dataset)
par

{'nproc': 8,
 'sextractor_flags': 4,
 'model_prediction': 0.8,
 'max_flux_threshold': 0.3,
 'elongation': 1.5,
 'annular_bin': 4,
 'flag_rim': 0,
 'min_acceptable_flux': 20000,
 'min_fwhm': 4.0,
 'max_fwhm': 10.0,
 'qfit_max': 5.0,
 'cfit_max': 5.0,
 'invert_east': [False, False],
 'invert_north': [False, False],
 'table1': 'sources_9319.csv',
 'table2': 'sources_9320.csv',
 'table1_calib': 'sources_calib_9319.csv',
 'table2_calib': 'sources_calib_9320.csv',
 'table_matched': 'table_match_9319_9320.fits',
 'table_non_matched': 'table_nomatch_9319_9320.fits',
 'table_psf_nonmatched': 'table_psf_nomatch_9319_9320.fits',
 'image1': 'GS00768_x.fits',
 'image2': 'GS00769_x.fits'}

## Read source tables generated by APPLAUSE database

From both **applause_dr4.source** and **applause_dr4.source_calib**. We need both, but we cannot get a joined version directly from APPLAUSE. Maybe they are limiting the output size. In any way, we get both and join them locally here.

In [4]:
table_src_1 = Table.read(fname(par['table1']), format='ascii.csv')
table_src_2 = Table.read(fname(par['table2']), format='ascii.csv')

table_calib_1 = Table.read(fname(par['table1_calib']), format='ascii.csv')
table_calib_2 = Table.read(fname(par['table2_calib']), format='ascii.csv')

In [5]:
table_1 = join(table_calib_1, table_src_1, keys='source_id')
table_2 = join(table_calib_2, table_src_2, keys='source_id')

In [6]:
print(len(table_1), len(table_2))

151083 149379


In [7]:
table_1

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349380000001,34938,12209,9319,102,0,9,5639.7905,3.401241,16,1.7833301e-32,0.9547166195413297,55.595146543415524,0.0968428,0.0834128,116.19279638684111,-6.652284001846378,30.80659271017268,48.87573004153747,0.5649584552501794,0.009414744970332244,0.82506563764253,47848,765572,13.764889,17.019691,1.0456479,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(0.0166629484345062 , 0.970318355311346)",34938,12209,9319,102,1,1981.2966,2.09694866605748,5.599326,0.4896937,-0.21710949,0.25392026,0.022170555,-0.22048235,11.434343,1981,3,0,1981.2966,3.401241,0.25392026,0.022170555,-0.22048235,NULL,NULL,NULL,NULL,NULL,8.912098,0.015099465,2723709.5,37869.758,11.8073635,0.04905693,189258.14,8549.192,5220.535,11.926231,37,6.0827627,-2946.5583,16,5639.7905,3.401241,9,1,0,0,1.7833301e-32,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380000002,34938,12209,9319,102,0,9,4418.4404,3.258278,17,7.2918836e-32,359.83372973563047,55.61027902578035,0.0968428,0.0834128,115.5703005451683,-6.514978507996677,30.05245588402188,49.27228790327279,0.5648165883464343,-0.0016390817593808333,0.8252148416865032,226685,3626974,7.1868157,17.647688,1.0492189,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(6.28028334361818 , 0.970582466952612)",34938,12209,9319,102,2,4482.26,1.9539996824477597,7.8481503,0.43700832,0.17823699,0.33551568,0.018726936,0.17961313,17.958813,4483,3,0,4482.26,3.258278,0.33551568,0.018726936,0.17961313,NULL,NULL,NULL,NULL,NULL,8.543116,0.011961347,3826077.8,42140.926,11.676017,0.04631115,213596.0,9108.542,5166.785,18.241182,42,6.4807405,-3785.3176,17,4418.4404,3.258278,9,1,0,0,7.2918836e-32,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380000003,34938,12209,9319,102,0,9,5602.6445,11.423781,0,0.50449014,0.9335739495525104,55.59754913562532,0.0968428,0.0834128,116.18142452652452,-6.647712902829138,30.794334523762917,48.884868540847926,0.564927297412249,0.009205705749936007,0.8250893306910094,47848,765569,31.244658,17.031

## Remove undesirable sources

Use a variety of criteria from SEXTRACTOR to get rid of possible contaminants.

We want to clean the first table only, since only well-defined star images from the first image should be present in our sample. The second table is taken as is; we want to see everything that is near the sources selected from the firsr table. 

In [8]:
table_1copy = table_1.copy()
table_2copy = table_2.copy()

# Extraction flags 
mask = table_1copy['sextractor_flags_1'] <= par['sextractor_flags']
table_1copy = table_1copy[mask]

# Likelihood of being valid star image 
mask = table_1copy['model_prediction_1'] > par['model_prediction']
table_1copy = table_1copy[mask]

# Round objects 
mask = table_1copy['elongation'] < par['elongation']
table_1copy = table_1copy[mask]

# Ring
mask = table_1copy['annular_bin_1'] <= par['annular_bin']
table_1copy = table_1copy[mask]

# Is source at plate rim ?
mask = table_1copy['flag_rim'] == par['flag_rim']
table_1copy = table_1copy[mask]

# flux threshold - set it to fraction of max peak in image
flux_threshold = max(table_1copy['flux_max']) * par['max_flux_threshold']
mask = table_1copy['flux_max'] > flux_threshold
table_1copy = table_1copy[mask]

print(len(table_1copy), len(table_2copy))

30180 149379


## Remove scanner artifacts

When plates were scanned twice or more, usually in two directions perpendicular to each other, all scans are included in a single table. Stars and artifacts *on* the plate will share the same celestial coordinates in both scans, but scanner-induced artifacts won't, because they move in x,y when the plate is rotated 90 deg. Thus we remove all rows that have no duplicates.  

To expedite the search, we split the job among all available performance CPUs.

In [9]:
# number of processors
nproc = par['nproc']

In [10]:
# The matching code lives in file library.py - it has to be kept in a separate 
# file because of restrictions in name space imposed by the parallelization library.

# Here, the parallelized code stores its product, the list of indices that have duplicates.
matched = []

# callback function to collect results from parallel workers
def collect_result(results):
    matched.extend(results)
    
results = []
pool = Pool(nproc)

row_range = int(len(table_1copy) / nproc)

for p in range(nproc):
    # workers are defined over ranges of rows in the table_1copy table
    worker = Worker2("w"+str(p), table_1copy, int(p*row_range), int((p+1)*row_range))

    r = pool.apply_async(worker, callback=collect_result)
    results.append(r)

for r in results:
    r.wait()

pool.close()

Worker  w0  -  0 3772
Worker  w1  -  3772 7544
Worker  w2  -  7544 11316
Worker  w3  -  11316 15088
Worker  w4  -  15088 18860
Worker  w5  -  18860 22632
Worker  w6  -  22632 26404
Worker  w7  -  26404 30176
w3  -  4% .  11500 22205 40349380049703     40349390034091    0.07710697166203317 0.06921211689530082
w1  -  6% .  4000 28186 40349380024227     40349390051637    0.1622420704279648 0.12783505515585603
w2  -  12% .  8000 24285 40349380038122     40349390040007    0.008418631909989926 0.009638807588885356
w3  -  18% .  12000 26655 40349380051620     40349390046783    0.12623853931472695 0.033224886675498055
w0  -  13% .  500 19019 40349380009599     40349390025117    0.4650566421560143 0.03645880400142687
w3  -  31% .  12500 22868 40349380053589     40349390035988    0.21352419050799654 0.0038296108044733046
w1  -  19% .  4500 17664 40349380025996     40349390021075    0.28654022412410995 0.06793583104069967
w3  -  44% .  13000 29044 40349380055620     40349390054488    0.2871980635

In [11]:
print(len(matched), " rows have duplicates")

14025  rows have duplicates


In [12]:
# if a small number of duplicates, or none, was detected, ignore them.
if len(matched) > len(table_1copy) / 10:

    # lots of duplicates: keep only the duplicated rows
    table_1copy = table_1copy[matched]

print(len(table_1copy))

14025


## Look for matches between 1st and 2nd tables

In [13]:
# Same code as above (can't be put in a function to avoid duplication).
#
# It generates the list of indices (in the first table) of all objects 
# for which it could find at least one matching entry in the second table.
matched = []

results = []
pool = Pool(nproc)

row_range = int(len(table_1copy) / nproc)

for p in range(nproc):
    # workers are defined over ranges of rows in the table_1copy table
    worker = Worker("w"+str(p), table_1copy, table_2copy, int(p*row_range), int((p+1)*row_range))

    r = pool.apply_async(worker, callback=collect_result)
    results.append(r)

for r in results:
    r.wait()

pool.close()

Worker  w0  -  0 1753
Worker  w1  -  1753 3506
Worker  w2  -  3506 5259
Worker  w3  -  5259 7012
Worker  w4  -  7012 8765
Worker  w5  -  8765 10518
Worker  w6  -  10518 12271
Worker  w7  -  12271 14024
w0  -  0% .  0 48513 40349380049116     40349400048514    0.48597127295124665 0.4127814224062831
w6  -  27% .  11000 9693 40349380009913     40349400009694    0.2608307696164047 0.013765383226882477
w7  -  13% .  12500 16937 40349380017339     40349400016938    0.17471910628046317 0.06365207722183186
w5  -  13% .  9000 30521 40349380031089     40349400030522    0.38557080922601017 0.052669148428208246
w3  -  13% .  5500 43093 40349380043743     40349400043094    0.5031825225159992 0.318632001858532
w4  -  27% .  7500 24997 40349380025401     40349400024998    0.6173165770178457 0.31896026765707575
w6  -  56% .  11500 12501 40349380012785     40349400012502    0.16302421249747567 0.2175964515515716
w1  -  14% .  2000 57300 40349380057831     40349400057301    0.11773545256801299 0.1729243

In [14]:
print(len(matched), " sources detected in 1st plate with a match in 2nd plate")

13988  sources detected in 1st plate with a match in 2nd plate


## Get the non-matched objects

Remove the matched rows from the first table. Whatever remains, is the table of objects for which a match couldn't be found.

Remove duplicated indices first, and save original table since removal is done in-place.

In [15]:
table_3 = table_1copy.copy()

m = list(dict.fromkeys(matched))
table_3.remove_rows(m)

In [16]:
table_3

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349380049749,34938,12209,9319,102,0,1,1250.612,3335.4937,0,1.0,359.7004097124922,56.91710661093143,0.0968428,0.0834128,115.76366476838677,-5.220265643296449,31.21061615930761,50.34699064777934,0.5458443597997983,-0.0028541571549151688,0.837881727131988,226813,3629009,13.722612,17.618925,1.049052,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(6.27795647025558 , 0.993390911069384)",34938,12209,9319,102,49749,4839.25,5172.498264939447,1.8430362,1.5482224,-36.09808,0.039377335,0.033299033,-33.30857,1.1904207,4839,5172,0,4839.25,5172.5063,0.039377335,0.033299033,-33.30857,NULL,NULL,NULL,NULL,NULL,10.129967,0.031442277,887183.1,25686.057,10.598527,0.01815991,576221.3,9635.476,27906.65,2.8575315,37,6.0827627,896.25507,0,1250.612,3335.4937,1,0,0,1,1.0,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380050009,34938,12209,9319,102,0,3,2845.9915,3002.8276,0,1.0,0.5524371329881319,56.91388265377987,0.0968428,0.0834128,116.22094516463082,-5.3150472719758834,31.776810871096725,50.05542912446628,0.5458735927920557,0.0052633927031881505,0.8378510114510485,47868,765902,12.321559,17.154264,1.0464005,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(0.00964184688092068 , 0.993334642402147)",34938,12209,9319,102,50009,3002.8276,5197.134402069652,2.0523188,1.7397393,-38.93845,0.035164814,0.030379647,-44.994102,1.1796703,3003,5197,0,3002.8276,5197.142,0.035164814,0.030379647,-44.994102,NULL,NULL,NULL,NULL,NULL,10.151687,0.024002116,869611.56,19219.629,10.269172,0.015022721,780425.5,10795.686,28447.234,2.538284,53,7.28011,2399.8057,0,2845.9915,3002.8276,3,0,0,1,1.0,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380050883,34938,12209,9319,102,0,2,2334.2837,3210.6587,0,1.0,0.27650032434854666,56.94287941436541,0.0968428,0.0834128,116.07816761345107,-5.2575907207544,31.62066594142601,50.17157691972454,0.5454685186770176,0.002632364799793251,0.838127177573822,47870,765922,21.231281,17.303795,1.0472448,NULL,NULL,NULL,NULL,NULL,N

## Write result to FITS table

In [17]:
table_3.write(fname(par['table_non_matched']), format='fits', overwrite=True)

In [18]:
# table with the matched objects only (selected by index array m)

table_1copy[m].write(fname(par['table_matched']), format='fits', overwrite=True)

In [19]:
table_1copy[m]

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349380005834,34938,12209,9319,102,1,4,3455.7747,843.1975,0,1.0,359.54564524459585,55.82463249932918,0.0968428,0.0834128,115.45446027567172,-6.272025130098838,30.05647036433907,49.541139201216815,0.5617100869692598,-0.004454446753235216,0.8273221477158151,226773,3628376,30.018616,17.785618,1.0500233,16.39332,0.37801373,16.232145,0.37802577,15.325736,0.37802577,16.260504,0.13281582,0.10922241,0,1,1.1778167,16.432016,0.6390662,1994721114513901440,NULL,0.906409,0.5825469,2,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(6.27525532072569 , 0.974323640829125)",34938,12209,9319,102,5834,5132.298,843.1290418920194,2.6360366,2.334493,80.52436,0.035576355,0.030179799,77.901405,1.1291689,5132,843,0,5132.298,843.1975,0.035576355,0.030179799,77.901405,NULL,NULL,NULL,NULL,NULL,9.562338,0.016191592,1496458.8,22311.28,9.733238,0.011998036,1278511.2,14124.884,25251.396,3.5143783,97,9.848858,3771.4834,0,3455.7747,843.1975,4,0,0,1,1.0,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380005839,34938,12209,9319,102,1,4,3488.1438,843.88074,2,1.0,359.6259104417735,55.824246599493705,0.0968428,0.0834128,115.49877096858349,-6.281671690965357,30.111002159720538,49.513203472254105,0.5617213480558415,-0.0036675838729824544,0.8273183643355578,226773,3628376,7.109186,17.740814,1.0497612,16.431149,0.3745515,16.290749,0.3745607,15.501173,0.3745607,16.292032,0.13911536,0.53985596,0,1,1.1778167,16.971004,0.6390662,1994720972772060416,NULL,0.789576,0.5625991,1,2022-06-13 20:53:25.680547+00,2022-06-13 20:53:25.680547+00,"(6.2766562126912 , 0.974316905606413)",34938,12209,9319,102,5839,4954.355,843.8123823285406,3.4660852,2.3269467,23.471306,0.057361223,0.035622746,23.281141,1.4895421,4954,844,0,4954.355,843.88074,0.057361223,0.035622746,23.281141,NULL,NULL,NULL,NULL,NULL,9.581128,0.026417574,1470783.0,35777.64,9.892748,0.014633976,1103826.2,14874.187,19092.309,4.085331,100,10.0,4098.813,2,3488.1438,843.88074,4,0,0,1,1.0,2022-06-13 05:05:13.513042+00,2022-06-13 05:05:13.513042+00
40349380005874,34938,12209,9319,102,0,4,3473.2476,814.16675,2,1.0,359.078105308039,55.81952383724266,0.0968428,0.0834128,115.195042850582,-6.22193581649423,